# Crear Target

## Ejercicio 1:
Cree el archivo `competencia_01.csv`, usando el `competencia_01_crudo.csv` con una variable adicional llamada `clase_ternaria`, que contenga las categorías **CONTINUA**, **BAJA+1** y **BAJA+2** 

### Ayudita 

Para practicar el muy útil y necesario lenguaje **SQL**, vamos a utilizar una base de datos **OLAP** llamada **DuckDB**. 

La documentación la puede encontrar [aquí](https://duckdb.org/docs/archive/0.8.1/sql/introduction)
Procedemos a instalarla, esto se debe ejecutar una sola vez

In [1]:
# %%bash 
# pip install duckdb
# pip install jupysql
# pip install duckdb-engine

Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb** 

In [2]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

Y ya podemos usar **SQL** dentro de una notebook!

In [3]:
%%sql
SELECT
    'hola mundo' 

,'hola mundo'
0,hola mundo


Para cargar el archivo `.csv` a una tabla:

In [4]:
%%sql
CREATE OR REPLACE TABLE competencia_01 AS
SELECT
    *
FROM read_csv_auto('../../datasets/raw/competencia_01_crudo.csv')

,Success


Hagamos unas queries básicas para comprobar que todo esta funcionando bien.

In [5]:
%%sql
SELECT
    *
FROM competencia_01
LIMIT 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,29183981,202103,1,0,0,50,197,14468.81,125765.29,2389.82,...,0.0,7,0.00,-114954.00,0.00,5938,101050.66,68,0,71811.06
1,29184630,202103,1,0,0,59,322,11901.57,74158.93,18750.68,...,0.0,1,0.00,-40330.15,17.59,4089,26834.09,7,0,3894.36
2,29185433,202103,1,0,0,68,268,847.15,21672.47,481.62,...,0.0,21,4692.00,-1173.00,0.00,7829,1651.36,3,0,1560.09
3,29185587,202103,1,0,0,79,322,4976.94,47735.98,1839.31,...,0.0,1,0.00,-15988.67,0.00,7580,30025.29,11,0,1700.85
4,29185646,202103,1,0,0,60,257,2860.45,37800.71,4035.40,...,0.0,21,380616.14,-97383.25,0.00,7827,359610.70,31,0,15600.90


In [6]:
%%sql
SELECT
    foto_mes
    , COUNT(*) AS cantidad -- cuenta cuantos casos hay en cada foto_mes 
                           -- y lo guarda en un campo llamado cantidad
FROM competencia_01
GROUP BY foto_mes

,foto_mes,cantidad
0,202103,163324
1,202104,163637
2,202105,164102


In [7]:
%%sql
SELECT
    numero_de_cliente,
    COUNT(*) AS cantidad
FROM competencia_01
GROUP BY numero_de_cliente
ORDER BY cantidad ASC
LIMIT 5

,numero_de_cliente,cantidad
0,50791576,1
1,51535131,1
2,51536728,1
3,52147925,1
4,52769507,1


Perfecto, ahora cree una nueva tabla con la variable adicional que se le pide. Algunas funciones que le pueden ser útiles:  [where](https://duckdb.org/docs/sql/query_syntax/where), [left join](https://duckdb.org/docs/sql/query_syntax/from), [case statement](https://duckdb.org/docs/sql/expressions/case)



**Paso 1:** Calcular la fecha máxima y mínima de cada usuario, para rellenar los faltantes en medio (si los hay)

In [8]:
%%sql
SELECT
    DISTINCT numero_de_cliente,
    MIN(foto_mes) AS min_foto_mes,
    MAX(foto_mes) AS max_foto_mes
FROM competencia_01
GROUP BY numero_de_cliente
LIMIT 5;

,numero_de_cliente,min_foto_mes,max_foto_mes
0,86584157,202103,202105
1,86694269,202103,202105
2,86724448,202103,202105
3,86815482,202103,202105
4,87204962,202103,202105


**Paso 2:** 

In [9]:
%%sql
WITH cte AS (
SELECT
    numero_de_cliente,
    MAX(foto_mes) AS max_foto_mes,
    MIN(foto_mes) AS min_foto_mes,
    MAX(foto_mes) - MIN(foto_mes) +1 AS cantidad_fotos,
    COUNT(*) AS real_fotos
FROM competencia_01
GROUP BY numero_de_cliente
ORDER BY cantidad_fotos DESC)
SELECT
    numero_de_cliente,
    max_foto_mes,
    min_foto_mes,
    cantidad_fotos,
    real_fotos
FROM cte
WHERE cantidad_fotos <> real_fotos;

,numero_de_cliente,max_foto_mes,min_foto_mes,cantidad_fotos,real_fotos
0,95380426,202105,202103,3,2


In [10]:
%%sql
SELECT
    *
FROM competencia_01
WHERE numero_de_cliente = 95380426

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,95380426,202103,1,0,0,52,153,-316.31,7892.03,1186.27,...,0.0,1,0.0,-23836.69,0.0,4574,6218.31,5,0,2744.82
1,95380426,202105,1,0,0,53,155,2535.14,6710.08,1023.54,...,0.0,5,0.0,-22629.28,0.0,4635,6500.14,3,0,5442.72


In [11]:
%%sql
CREATE OR REPLACE TABLE competencia_01 AS (
    WITH RECURSIVE clientes_min_max_foto_mes AS (
    SELECT
        numero_de_cliente,
        MAX(foto_mes) AS max_foto_mes,
        MIN(foto_mes) AS min_foto_mes,
        MAX(foto_mes) - MIN(foto_mes) +1 AS cantidad_fotos,
        COUNT(*) AS real_fotos
    FROM competencia_01
    GROUP BY numero_de_cliente
    ORDER BY cantidad_fotos DESC
    ),
    ccf AS ( --ccf: clientes_con_faltantes
    SELECT
        numero_de_cliente,
        max_foto_mes,
        min_foto_mes,
        cantidad_fotos,
        real_fotos
    FROM clientes_min_max_foto_mes
    WHERE cantidad_fotos <> real_fotos
    ),
    FotoMesFaltantes AS (
    SELECT
        ccf.numero_de_cliente,
        min_foto_mes AS foto_mes_sin_faltante
    FROM ccf
    UNION ALL
    SELECT
        FotoMesFaltantes.numero_de_cliente,
        foto_mes_sin_faltante + 1
    FROM FotoMesFaltantes
    JOIN
        ccf ON foto_mes_sin_faltante + 1 <= ccf.max_foto_mes
    )
    SELECT
        c01.*,
        COALESCE(fmf.numero_de_cliente, c01.numero_de_cliente) AS numero_de_cliente_new,
        COALESCE(fmf.foto_mes_sin_faltante, c01.foto_mes) AS foto_mes_new
    FROM FotoMesFaltantes AS fmf
    FULL OUTER JOIN competencia_01 AS c01
    ON fmf.numero_de_cliente = c01.numero_de_cliente AND fmf.foto_mes_sin_faltante = c01.foto_mes
    ORDER BY numero_de_cliente_new, foto_mes_new ASC
    )
;

,Success


In [12]:
%%sql
SELECT
    COUNT(*) - COUNT(numero_de_cliente) AS cantidad_faltantes_cliente,
    COUNT(*) - COUNT(foto_mes) AS cantidad_faltantes_foto_mes,

    COUNT(*) - COUNT(numero_de_cliente_new) AS cantidad_faltantes_cliente_new,
    COUNT(*) - COUNT(foto_mes_new) AS cantidad_faltantes_foto_mes_new
FROM competencia_01
LIMIT 100

,cantidad_faltantes_cliente,cantidad_faltantes_foto_mes,cantidad_faltantes_cliente_new,cantidad_faltantes_foto_mes_new
0,1,1,0,0


In [13]:
%%sql
SELECT
    *
FROM competencia_01

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,numero_de_cliente_new,foto_mes_new
0,29183981.0,202103.0,1.0,0.0,0.0,50.0,197.0,14468.81,125765.29,2389.82,...,0.0,-114954.00,0.00,5938.0,101050.66,68.0,0.0,71811.06,29183981,202103
1,29183981.0,202104.0,1.0,0.0,0.0,50.0,198.0,10745.55,140069.00,2326.56,...,0.0,-71811.06,0.00,5968.0,133279.52,79.0,0.0,50368.62,29183981,202104
2,29183981.0,202105.0,1.0,0.0,0.0,50.0,199.0,15510.89,153502.25,2699.83,...,87975.0,-50368.62,0.00,5999.0,114271.45,83.0,0.0,65066.31,29183981,202105
3,29184630.0,202103.0,1.0,0.0,0.0,59.0,322.0,11901.57,74158.93,18750.68,...,0.0,-40330.15,17.59,4089.0,26834.09,7.0,0.0,3894.36,29184630,202103
4,29184630.0,202104.0,1.0,0.0,0.0,59.0,323.0,-2219.25,67723.98,3709.88,...,0.0,-52170.68,17.59,4119.0,26117.22,7.0,0.0,5524.83,29184630,202104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491059,186790659.0,202105.0,0.0,0.0,0.0,35.0,1.0,0.00,0.00,0.00,...,0.0,NaN,NaN,28.0,NaN,NaN,NaN,0.00,186790659,202105
491060,186792315.0,202105.0,1.0,0.0,0.0,37.0,1.0,51.81,51.81,0.00,...,0.0,NaN,NaN,25.0,NaN,NaN,NaN,0.00,186792315,202105
491061,186795526.0,202105.0,1.0,0.0,0.0,47.0,1.0,327.75,327.75,0.00,...,0.0,NaN,NaN,21.0,NaN,NaN,NaN,0.00,186795526,202105
491062,186798379.0,202105.0,0.0,0.0,0.0,32.0,1.0,0.00,0.00,0.00,...,0.0,NaN,NaN,11.0,NaN,NaN,NaN,0.00,186798379,202105


In [14]:
%%sql
ALTER TABLE competencia_01
DROP COLUMN numero_de_cliente;

ALTER TABLE competencia_01
DROP COLUMN foto_mes;

ALTER TABLE competencia_01
RENAME COLUMN numero_de_cliente_new TO numero_de_cliente;

ALTER TABLE competencia_01
RENAME COLUMN foto_mes_new TO foto_mes;

,Success


In [15]:
%%sql
CREATE OR REPLACE TABLE competencia_01 AS (
    SELECT
        *,
        CASE
            WHEN foto_mes = (SELECT MAX(foto_mes) FROM competencia_01 WHERE numero_de_cliente = c.numero_de_cliente) THEN 'BAJA+2'
            WHEN foto_mes = (SELECT MAX(foto_mes) - 1 FROM competencia_01 WHERE numero_de_cliente = c.numero_de_cliente) THEN 'BAJA+1'
            ELSE 'CONTINUA'
        END AS clase_ternaria
    FROM
        competencia_01 c
    )
;

,Success


In [16]:
%%sql
SELECT
    numero_de_cliente,
    foto_mes,
    clase_ternaria
FROM competencia_01
WHERE numero_de_cliente IN (95380426, 29183981, 29199353)

,numero_de_cliente,foto_mes,clase_ternaria
0,95380426,202103,CONTINUA
1,95380426,202104,BAJA+1
2,95380426,202105,BAJA+2
3,29199353,202105,BAJA+2
4,29183981,202103,CONTINUA
5,29183981,202104,BAJA+1
6,29183981,202105,BAJA+2


In [17]:
%%sql
SELECT
    *
FROM competencia_01
LIMIT 5;

,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,...,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,numero_de_cliente,foto_mes,clase_ternaria
0,0,0,0,62,1,1.23,1.23,0.00,1.05,0.00,...,NaN,NaN,14,NaN,NaN,NaN,0.0,60947729,202103,CONTINUA
1,1,0,0,62,2,274.90,276.13,283.04,-865.47,858.53,...,NaN,NaN,44,NaN,NaN,NaN,0.0,60947729,202104,BAJA+1
2,1,0,0,62,3,371.85,647.98,283.04,-1147.33,1223.04,...,NaN,NaN,75,NaN,NaN,NaN,0.0,60947729,202105,BAJA+2
3,1,0,0,77,270,1396.04,32770.10,1900.82,-1308.44,878.11,...,-36809.50,0.0,4088,22077.51,10.0,0.0,3988.2,60948011,202103,CONTINUA
4,1,0,0,78,271,1232.00,31468.10,2053.14,-1497.91,797.87,...,-36973.47,0.0,4118,26163.84,12.0,0.0,3988.2,60948011,202104,BAJA+1


## Ejercicio 1.1

* ¿Cuál es la nominalidad de cada clase?
* ¿Cuál es la proporción del target?

In [18]:
%%sql
select
    clase_ternaria,
    count(*) as cant,
    cant/(SELECT count(*) from competencia_01) as prop_tot,
    cant/(SELECT count(*) from competencia_01 WHERE clase_ternaria is not NULL) as prop_m0
from competencia_01
group by clase_ternaria

,clase_ternaria,cant,prop_tot,prop_m0
0,CONTINUA,161312,0.328495,0.328495
1,BAJA+1,163632,0.333219,0.333219
2,BAJA+2,166120,0.338286,0.338286


Para guardar a un **.csv** simplemente debe ejecutar la siguiente sentencia

In [19]:
%%sql
COPY competencia_01 TO '../../datasets/interim/competencia_01.csv' (FORMAT CSV, HEADER)

,Success


## Ejercicio 2 - Avanzado
Use ahora el archivo `ejercicio_target.csv` y calcule para todos los clientes en todos los periodos su **clase_ternaria** al mismo tiempo